In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import procore_formatter
import excel_creator
import metrics

In [2]:
raw = pd.read_csv('submittal_logs.csv')

In [3]:
df = procore_formatter.formatter(raw)
df['Rev.'] = df['Rev.'].astype(np.int)

C:\Users\Kam Fong\AppData\Local\Temp\ipykernel_14740\3212432046.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['Rev.'] = df['Rev.'].astype(np.int)


In [4]:
df.to_excel('test.xlsx')

In [5]:
def engineer(contractor):

    engineers = {
        'Ronny F' : ['Fred Geller Electric', 
                       'Corporate Electric',
                       'Societe International de Telecommunications Aeronautiques',
                       'Boingo Wireless',
                       'Schneider Electric',
                       'SITA',
                       'Iconics'],

        'Paul M' : ['B&G Electric',
                       'E-J Electric Installation Co.'],

        'Andy K' : ['FCS Group',
                       'Cassone Leasing, Inc.',
                       'JC Elite Construction',
                       'Lafayette Glass',
                       'Component Assembly Systems',
                       'Consolidated Carpet Trade Workroom',
                       'Garcia Marble and Tile',
                       'McKeon Door East'],

        'Nancy A' : ['Center Sheet Metal',
                        'PJ Mechanical LLC.',
                        'Olympic Plumbing And Heating Services Inc.',
                        'Rael Automatic Sprinkler'
                        ],

        'Brett Z' : ['Schindler Elevator Corp.',
                            'Monpat Construction Inc.',
                            'Aero Bridge Works',
                            'John Bean Technologies Corporation',
                            'ILC Dover',
                            'W&W Glass, LLC.',
                            'Commodore Construction Corp.',
                            'Island International Enterprises',
                            'Yorie Tile & Terrazzo'
                            ],

        'Rebecca J' : ['Curtis Partition Corp.',
                        'Long Island Concrete',
                        'Post Road Iron Works Inc.',
                        'Long Island Concrete',
                        'Going Sign & Servicing Co. Inc.',
                        'Architectural Casework, Inc.',
                        'Image Manufacturing Group',
                        'Office Images',
                        'Flockheart',
                        'Nanolumens'
                       
                            ],
        'Andre F' : ['Peter Scalamandre & Sons Inc',
                     'Owen Steel Company, Inc.',
                     'Falco Construction Corp.'
                  ]
    }
    
    for key, value in engineers.items():
        if contractor == value:
            return key
        
        elif isinstance(value, list) and contractor in value:
            return key
        
    return 'Miscellaneous'

In [6]:
df['Responsible Contractor'] = df['Responsible Contractor'].str.replace('/', ' ')
contractors = df['Responsible Contractor'].unique()

In [7]:
closeout = ['As-Built',
           'Attic Stock',
           'Certificate of Substantial Completion',
           'Certificate of Final Completion',
           'Closeouts',
           'O&M Manual',
           'Product Manual',
           'Record Documents',
           'Spare Parts List',
           'Training Material',
           'Warranty / Guarantee']

types = df['Description'].unique()
not_closeout = [x for x in types if x not in closeout]

default_font = 'Calibri'

In [8]:
today = datetime.datetime.today()
date_folder = today.strftime("%Y-%m-%d")
directory =f'{date_folder} CIL Logs'
if not os.path.exists(directory):
    os.mkdir(directory)

In [9]:
rev_response = ['Reasons Given', 'LEED Only', 'and Resubmit']

df1 = raw.copy()
df1.loc[:, 'word'] = df1.loc[:, 'Response'].str.split().str[-2:]
df1['word'] = df1.loc[:, 'word'].str.join(' ')
rev = df1[df1['word'].isin(rev_response) &
         df1['Status'].isin(['Closed', 'Void', 'Superseded'])].copy()

In [17]:

for i in range(len(contractors)):
    
    data = df[df['Responsible Contractor'] == contractors[i]]
    name = engineer(contractors[i])
    
    name_directory = directory + f'/{name}'
    if not os.path.exists(name_directory):
        os.mkdir(name_directory)
        
    writer = pd.ExcelWriter(f'{name_directory}/{contractors[i]} CIL Log {date_folder}.xlsx', 
                        engine = 'xlsxwriter',
                        date_format = 'mm-dd-yyyy', 
                        datetime_format='mm-dd-yyyy')
    workbook = writer.book
    
    data.to_excel(writer,
                 sheet_name = 'Submittals Master Log',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['Submittals Master Log']
    excel_creator.generate_worksheet(data, name, contractors[i], workbook, submittal_log)
    
    number_total_format = workbook.add_format({
    'bold' : True,
    'font_color' : 'black', 
    'font_size' : 11,
    'font_name' : default_font, 
    'align' : 'center',
    'valign' : 'vcenter', 
    'border' : 1,
    'num_format' : '#,##0'
    })

    percent_format = workbook.add_format({
        'bold' : False,
        'font_color' : 'black', 
        'font_size' : 11,
        'font_name' : default_font, 
        'align' : 'center',
        'valign' : 'vcenter', 
        'border' : 1,
        'num_format' : '0.00%'
    })
    
    submittal_log.write('J1', 'North #')
    submittal_log.write('K1', 'North %')
    submittal_log.write('L1', 'South #')
    submittal_log.write('M1', 'South %')
    submittal_log.write('I2', 'Total')
    submittal_log.write('I3', 'Open')
    submittal_log.write('I4', 'Closeout')
    submittal_log.write('I5', 'Design Team')
    submittal_log.write('I6', 'R&R')
    submittal_log.write('I7', 'R&R LEED')
    submittal_log.write('I8', 'Closed')
    
    north_metrics = metrics.metrics(data, ['N', 'N/S'])
    total = 0
    for key in north_metrics:
        total = total + north_metrics[key][1]
    
    submittal_log.write('J2', total, number_total_format)
    if total == 0:
        total = 1 
        
    row = 2
    for key in north_metrics:
        submittal_log.write(row, 9, north_metrics[key][1], number_total_format)
        submittal_log.write(row, 10, north_metrics[key][1]/total, percent_format)
        row = row + 1
        
        
    south_metrics = metrics.metrics(data, ['S'])
    total = 0
    for key in south_metrics:
        total = total + south_metrics[key][1]

        
    submittal_log.write('L2', total, number_total_format)
    if total == 0:
        total = 1  
        
    row = 2
    for key in south_metrics:
        submittal_log.write(row, 11, south_metrics[key][1], number_total_format)
        submittal_log.write(row, 12, south_metrics[key][1]/total, percent_format)
        row = row + 1
        
     
    
    #BIC Design Team
    pending = data[(data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                (data['Ball In Court'].str.contains('P022', na = False))]
    pending.to_excel(writer,
                 sheet_name = 'BIC Design Team',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['BIC Design Team']
    excel_creator.generate_worksheet(pending, name, contractors[i], workbook, submittal_log)    
    
    #North Open Submittals
    north = data[(data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                (data['Phase'].str.contains('N', na = False)) &
                (~data['Ball In Court'].str.contains('P022', na = False))]
    north.to_excel(writer,
                 sheet_name = 'North Open Items',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['North Open Items']
    excel_creator.generate_worksheet(north, name, contractors[i], workbook, submittal_log)
    
    
    #South Open Submittals
    south = data[(data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                (data['Phase'].str.contains('S', na = False)) &
                (~data['Ball In Court'].str.contains('P022', na = False))]
    south.to_excel(writer,
                 sheet_name = 'South Open Items',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['South Open Items']
    excel_creator.generate_worksheet(south, name, contractors[i], workbook, submittal_log)
    
    
    #R&R Submissions 
    revise = data[(data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                (data['Rev.'] != 0) &
                (~data['Ball In Court'].str.contains('P022', na = False))]
    revise.to_excel(writer,
                 sheet_name = 'R&R Submittals',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['R&R Submittals']
    excel_creator.generate_worksheet(revise, name, contractors[i], workbook, submittal_log)
    
    
    
    #Missed Submissions
    submit_by = data[(pd.to_datetime(data['Submit By Date']) < today) & 
                    (data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                    (~data['Ball In Court'].str.contains('P022', na = False))]
    submit_by.to_excel(writer,
                 sheet_name = 'Missed Submissions',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['Missed Submissions']
    excel_creator.generate_worksheet(submit_by, name, contractors[i], workbook, submittal_log)
    

    
        
    upcoming = data[(pd.to_datetime(data['Submit By Date']) > today) & 
                    (pd.to_datetime(data['Submit By Date']) < (today + datetime.timedelta(days=28))) &
                    (data['Status'].isin(['Open', 'Pending', 'Draft']))]
    upcoming.to_excel(writer,
                 sheet_name = '4 Week Lookahead',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['4 Week Lookahead']
    excel_creator.generate_worksheet(submit_by, name, contractors[i], workbook, submittal_log)

    
    
#     df2 = rev[rev['Responsible Contractor'] == contractors[i]]
#     add_revision = df2.iloc[0:0]
    
#     for j in range(len(df2)):
#     #print(rev.iloc[i, 1], rev.iloc[i, 6])
#         title = df2.iloc[j, 1]
#         revision = df2.iloc[j, 6]
#         check = df1.loc[df1['#'] == title, 'Rev.']
#         if not np.isclose(check, revision + 1).any():
#             #print( f'{title} {revision}')
#             add_revision = pd.concat([add_revision, df2.iloc[[j]]])
            
#     add_revision = procore_formatter.formatter(add_revision)
    
    
    
#     add_revision.to_excel(writer,
#              sheet_name = 'R&R Missing',
#              index = False,
#              startrow = 8)
#     submittal_log = writer.sheets['R&R Missing']
#     excel_creator.generate_worksheet(add_revision, name, contractors[i], workbook, submittal_log)
    
    
    
#     df_co = data[data['Description'] == 'Closeouts']
#     df_co.to_excel(writer,
#                  sheet_name = '(Closeout) To be Corrected',
#                  index = False,
#                  startrow = 8)
#     submittal_log = writer.sheets['(Closeout) To be Corrected']
#     excel_creator.generate_worksheet(submit_by, name, contractors[i], workbook, submittal_log)
    
    
    
    
    writer.close()
    writer.handles = None

    print(contractors[i])

D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']


Peter Scalamandre & Sons Inc


D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']


Delta CMT


D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']


...

Architectural Casework, Inc.


D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']


Post Road Iron Works Inc.


D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
D:\PROGAMMING\Procore\New folder\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']


Nanolumens


In [11]:
def open_submittals(df, location, description):
    status = ['Open', 'Draft', 'Pending']
    
    open_df = df[df['Rev.'] == 0]
    open_df = open_df[open_df['Status'].isin(status) &
                  open_df['Location'].isin(location) &
                  ~open_df['Ball In Court'].str.contains('P022', na = False)]
    
    if description != 'all':
        open_df = open_df[open_df['Type'].isin(description)]
    
    return [open_df, open_df['Status'].count()]

In [12]:
test = df[df['Responsible Contractor'] == 'Fred Geller Electric']
test['Location'] = test['Phase']
test['Type'] = test['Description']

C:\Users\Kam Fong\AppData\Local\Temp\ipykernel_14740\2975486550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Location'] = test['Phase']
C:\Users\Kam Fong\AppData\Local\Temp\ipykernel_14740\2975486550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Type'] = test['Description']


In [13]:
test2 = open_submittals(test, ['S'], not_closeout)

In [14]:
test2[1]

24

In [15]:
    i = 1
    data = df
    name = "master"
    
    name_directory = directory + f'/{name}'
    if not os.path.exists(name_directory):
        os.mkdir(name_directory)
        
    writer = pd.ExcelWriter(f'{name_directory}/name CIL Log {date_folder}.xlsx', 
                        engine = 'xlsxwriter',
                        date_format = 'mm-dd-yyyy', 
                        datetime_format='mm-dd-yyyy')
    workbook = writer.book
    
    data.to_excel(writer,
                 sheet_name = 'Submittals Master Log',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['Submittals Master Log']
    excel_creator.generate_worksheet(data, name, contractors[i], workbook, submittal_log)
    
    number_total_format = workbook.add_format({
    'bold' : True,
    'font_color' : 'black', 
    'font_size' : 11,
    'font_name' : default_font, 
    'align' : 'center',
    'valign' : 'vcenter', 
    'border' : 1,
    'num_format' : '#,##0'
    })

    percent_format = workbook.add_format({
        'bold' : False,
        'font_color' : 'black', 
        'font_size' : 11,
        'font_name' : default_font, 
        'align' : 'center',
        'valign' : 'vcenter', 
        'border' : 1,
        'num_format' : '0.00%'
    })
    
    submittal_log.write('J1', 'North #')
    submittal_log.write('K1', 'North %')
    submittal_log.write('L1', 'South #')
    submittal_log.write('M1', 'South %')
    submittal_log.write('I2', 'Total')
    submittal_log.write('I3', 'Open')
    submittal_log.write('I4', 'Closeout')
    submittal_log.write('I5', 'Design Team')
    submittal_log.write('I6', 'R&R')
    submittal_log.write('I7', 'R&R LEED')
    submittal_log.write('I8', 'Closed')
    
    north_metrics = metrics.metrics(data, ['N', 'N/S'])
    total = 0
    for key in north_metrics:
        total = total + north_metrics[key][1]
    
    submittal_log.write('J2', total, number_total_format)
    if total == 0:
        total = 1 
        
    row = 2
    for key in north_metrics:
        submittal_log.write(row, 9, north_metrics[key][1], number_total_format)
        submittal_log.write(row, 10, north_metrics[key][1]/total, percent_format)
        row = row + 1
        
        
    south_metrics = metrics.metrics(data, ['S'])
    total = 0
    for key in south_metrics:
        total = total + south_metrics[key][1]

        
    submittal_log.write('L2', total, number_total_format)
    if total == 0:
        total = 1  
        
    row = 2
    for key in south_metrics:
        submittal_log.write(row, 11, south_metrics[key][1], number_total_format)
        submittal_log.write(row, 12, south_metrics[key][1]/total, percent_format)
        row = row + 1
        
     
    
    
    pending = data[(data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                (data['Ball In Court'].str.contains('P022', na = False))]
    pending.to_excel(writer,
                 sheet_name = 'Design Team Items',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['Design Team Items']
    excel_creator.generate_worksheet(pending, name, contractors[i], workbook, submittal_log)    
    
    
    north = data[(data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                (data['Phase'].str.contains('N', na = False)) &
                (~data['Ball In Court'].str.contains('P022', na = False))]
    north.to_excel(writer,
                 sheet_name = 'North Open Items',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['North Open Items']
    excel_creator.generate_worksheet(north, name, contractors[i], workbook, submittal_log)
    
    
    
    submit_by = data[(pd.to_datetime(data['Submit By Date']) < today) & 
                    (data['Status'].isin(['Open', 'Pending', 'Draft'])) &
                    (~data['Ball In Court'].str.contains('P022', na = False))]
    submit_by.to_excel(writer,
                 sheet_name = 'Missed Submissions',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['Missed Submissions']
    excel_creator.generate_worksheet(submit_by, name, contractors[i], workbook, submittal_log)
    

    
        
    upcoming = data[(pd.to_datetime(data['Submit By Date']) > today) & 
                    (pd.to_datetime(data['Submit By Date']) < (today + datetime.timedelta(days=28))) &
                    (data['Status'].isin(['Open', 'Pending', 'Draft']))]
    upcoming.to_excel(writer,
                 sheet_name = '4 Week Lookahead',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['4 Week Lookahead']
    excel_creator.generate_worksheet(submit_by, name, contractors[i], workbook, submittal_log)

    
    
    df2 = rev[rev['Responsible Contractor'] == contractors[i]]
    add_revision = df2.iloc[0:0]
    
    for j in range(len(df2)):
    #print(rev.iloc[i, 1], rev.iloc[i, 6])
        title = df2.iloc[j, 1]
        revision = df2.iloc[j, 6]
        check = df1.loc[df1['#'] == title, 'Rev.']
        if not np.isclose(check, revision + 1).any():
            #print( f'{title} {revision}')
            add_revision = pd.concat([add_revision, df2.iloc[[j]]])
            
    add_revision = procore_formatter.formatter(add_revision)
    
    
    
    add_revision.to_excel(writer,
             sheet_name = 'R&R Missing',
             index = False,
             startrow = 8)
    submittal_log = writer.sheets['R&R Missing']
    excel_creator.generate_worksheet(add_revision, name, contractors[i], workbook, submittal_log)
    
    
    
    df_co = data[data['Description'] == 'Closeouts']
    df_co.to_excel(writer,
                 sheet_name = '(Closeout) To be Corrected',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets['(Closeout) To be Corrected']
    excel_creator.generate_worksheet(submit_by, name, contractors[i], workbook, submittal_log)
    
    
    
    
    writer.close()
    writer.handles = None

    print(contractors[i])

Delta CMT


In [16]:
    writer.close()
    writer.handles = None

C:\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


AttributeError: 'NoneType' object has no attribute 'close'